<a href="https://colab.research.google.com/github/anaferreira744/DE-DP-ADF/blob/main/example_2_rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Usecase 2
- Reading data from "rate"
- Aggregating data by window time
- Checking results from query in memory

# Setting up PySpark

In [ ]:
%pip install pyspark

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').appName('Test streaming').getOrCreate()

# Write output in memory

In [2]:
import pyspark.sql.functions as F

# read stream
stream1 = spark.readStream.format("rate").option("rowsPerSecond", 10).load()

# transform
transformed = stream1.withColumn("minute", F.minute("timestamp"))
agg = transformed.groupBy(F.window(transformed.timestamp, "5 seconds")).count()

# write stream in memory
query = (agg.writeStream
.format('memory')
.queryName('my_query')
.outputMode('complete')
.start()
)

In [5]:
spark.sql("select * from my_query order by window desc").show(10,False)

+------------------------------------------+-----+
|window                                    |count|
+------------------------------------------+-----+
|{2024-11-23 15:47:40, 2024-11-23 15:47:45}|23   |
|{2024-11-23 15:47:35, 2024-11-23 15:47:40}|50   |
|{2024-11-23 15:47:30, 2024-11-23 15:47:35}|50   |
|{2024-11-23 15:47:25, 2024-11-23 15:47:30}|50   |
|{2024-11-23 15:47:20, 2024-11-23 15:47:25}|50   |
|{2024-11-23 15:47:15, 2024-11-23 15:47:20}|50   |
|{2024-11-23 15:47:10, 2024-11-23 15:47:15}|50   |
|{2024-11-23 15:47:05, 2024-11-23 15:47:10}|50   |
|{2024-11-23 15:47:00, 2024-11-23 15:47:05}|50   |
|{2024-11-23 15:46:55, 2024-11-23 15:47:00}|50   |
+------------------------------------------+-----+
only showing top 10 rows



In [ ]:
query.stop()

# Write output as parquet

In [ ]:
!rm -rf content/output

In [6]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame

def save_parquet(df, batch_id):
  (df
   .withColumn("batch_id",F.lit(batch_id))
   .withColumn("load_time",F.current_timestamp())
   .write.mode("append")
   .parquet("content/output/rate_parquet")
  )

# read stream
stream1 = spark.readStream.format("rate").option("rowsPerSecond", 10).load()

# transform
transformed = stream1.withWatermark("timestamp", "5 seconds").withColumn("minute", F.minute("timestamp"))
agg = transformed.groupBy(F.window(transformed.timestamp, "5 seconds")).count()

# write stream as parquet with foreachBatch
query = (agg.writeStream
.option('checkpointLocation', 'content/output/checkpoint')
.trigger(processingTime='20 seconds')
.outputMode('append')
.foreachBatch(save_parquet)
.start()
)


In [11]:
result = spark.read.format("parquet").load("content/output/rate_parquet/")
result.sort(F.asc("window")).show(100, False)

+------------------------------------------+-----+--------+--------------------------+
|window                                    |count|batch_id|load_time                 |
+------------------------------------------+-----+--------+--------------------------+
|{2024-11-23 15:55:20, 2024-11-23 15:55:25}|13   |2       |2024-11-23 15:56:12.414428|
|{2024-11-23 15:55:25, 2024-11-23 15:55:30}|50   |2       |2024-11-23 15:56:12.414428|
|{2024-11-23 15:55:30, 2024-11-23 15:55:35}|50   |2       |2024-11-23 15:56:12.414428|
|{2024-11-23 15:55:35, 2024-11-23 15:55:40}|50   |2       |2024-11-23 15:56:12.414428|
|{2024-11-23 15:55:40, 2024-11-23 15:55:45}|50   |3       |2024-11-23 15:56:36.292465|
|{2024-11-23 15:55:45, 2024-11-23 15:55:50}|50   |3       |2024-11-23 15:56:36.292465|
|{2024-11-23 15:55:50, 2024-11-23 15:55:55}|50   |3       |2024-11-23 15:56:36.292465|
|{2024-11-23 15:55:55, 2024-11-23 15:56:00}|50   |3       |2024-11-23 15:56:36.292465|
|{2024-11-23 15:56:00, 2024-11-23 15:56:05}

In [ ]:
query.stop()

# Enrich data with faker

In [12]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.2 MB/s eta 0:00:00


In [ ]:
!rm -rf content/output/events

In [13]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from faker import Faker

def insert_into_table(df, batch_id):
  fake = Faker()
  new_columns = {
      'name': F.lit(fake.name()),
      'address': F.lit(fake.address()),
      'email': F.lit(fake.email()),
      'dob': F.lit(fake.date_of_birth()),
      'phone': F.lit(fake.phone_number())
  }
  df = df.withColumns(new_columns)
  df.write.mode("append").format("parquet").save("content/output/events")

# read stream
df_stream = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# write stream
query = (df_stream.writeStream
.outputMode('append')
.trigger(processingTime='1 seconds')
.foreachBatch(insert_into_table)
.start()
)

In [17]:
query.stop()


ERROR:py4j.clientserver:There was an exception while executing the Python Proxy on the Python Side.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/utils.py", line 120, in call
    raise e
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/utils.py", line 117, in call
    self.func(DataFrame(jdf, wrapped_session_jdf), batch_id)
  File "<ipython-input-13-6dab22f90f5d>", line 15, in insert_into_table
    df.write.mode("append").format("parquet").save("content/output/events")
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/readwriter.py", line 1463, in save
    self._jwrite.save(path)
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "/usr/local/lib/python3.10/dist-packages/pyspa

In [16]:
spark.read.parquet("content/output/events").show(100, False)

+-----------------------+-----+----------------+---------------------------------------------------------+-------------------------------+----------+---------------------+
|timestamp              |value|name            |address                                                  |email                          |dob       |phone                |
+-----------------------+-----+----------------+---------------------------------------------------------+-------------------------------+----------+---------------------+
|2024-11-23 16:02:20.782|39   |Linda Anderson  |7407 Thomas Locks Apt. 617\nNorth Yvonnechester, ME 90143|simpsonnicole@example.org      |1959-03-30|759-390-9614x29026   |
|2024-11-23 16:02:21.782|40   |Linda Anderson  |7407 Thomas Locks Apt. 617\nNorth Yvonnechester, ME 90143|simpsonnicole@example.org      |1959-03-30|759-390-9614x29026   |
|2024-11-23 16:02:22.782|41   |Linda Anderson  |7407 Thomas Locks Apt. 617\nNorth Yvonnechester, ME 90143|simpsonnicole@example.org      |19

In [ ]:
query.stop()